<center><b>EE 538 Autumn 2020<br>
Analog Circuits for Sensor Systems<br>
University of Washington Electrical & Computer Engineering</b></center>


<b>Instructor: Jason Silver<br>
Assignment #3 (10 points)<br>
Due Sunday, October 25 (Submit on Canvas as a Jupyter Notebook)</b> 

*Please show your work*

<b>Problem 1: Common-source amplifier with emitter-follower output stage

<center><img src="img/HW3_gain_stage.png" width=400></center>

The MOSFET-based common-source amplifier can be combined with a BJT emitter-follower output stage to construct a circuit that provides voltage gain while taking advantage of the high input impedance of the MOSFET and low output impedance of the emitter follower.

For the following, $T = 27C$, $R_D = 5k\Omega$, $R_E = 2k\Omega$, $V_{IN,DC} = 0.9V$, $V_{DD} = 5V$, and $C_L = 1uF$

Use the Analog Devices MAT-02 $npn$ transistor ($V_A = 150V$, $\beta = 500$, $I_S = 0.6 pA$) for your analysis and simulations.

Use the Level 1 NMOS SPICE model provided with the assignment ($\mu = 350$, $C_{ox} = 384 nF/m^2$ $V_{th} = 0.7V$, $\lambda = 0.01 V^{-1}$, $W = 100\mu m$, $L = 1 \mu m$). 

<u>*Analysis*</u>

__a)__ *Estimate* the DC operating point of the circuit ($I_{D0}$, $I_{C0}$, $V_{1,DC}$, and $V_{OUT,DC}$) assuming $\lambda = 0$ for $M_1$ and $V_{BE} = 0.6V$, $I_B = 0$ for $Q_1$ (you are also welcome to calculate $V_{BE}$ using the iterative method discussed at the end of Lecture 3 if you'd like to try this approach).

*Solution:*

Calculate DC operating point of the circuit:

DC operating point of $M_1$ and ignore base current of $Q_1$:
$$I_{D0}=\frac{1}{2}\mu C_{ox}\frac{W}{L}(V_{IN,DC}-V_{th})^2$$
$$V_{1,DC}=V_{DD}-I_{D0}R_{D}$$

In [2]:
#%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

#Parameter value provided:
R_D = 5e3
u=350
Cox=384e-9
V_INDC=0.9
Vth=0.7
lam=0.01
W=100e-6
L=1e-6
VDD=5

#calculate DC operating point of M1
I_D0=0.5*u*Cox*W/L*(V_INDC-Vth)**2
V1_DC=VDD-I_D0*R_D

In [4]:
print('The DC current of M1 is', f'{I_D0:0.5}', 'A')
print('The DC voltage at V1 is', f'{V1_DC:0.3}', 'V')

The DC current of M1 is 0.0002688 A
The DC voltage at V1 is 3.66 V


DC operating point of $C1$ can be calculated by assuming $V_{BE}=0.6V$, also you can use the iterative method discussed in review session of Lecture 3 (More details can be found in DEMO2 slide on Canvas).

Now assuming $V_{BE}=0.6V$:
$$V_{OUT,DC}=V_{1,DC}-V_{BE}$$
$$I_{C0}=I_{S}(e^{\frac{V_{BE}}{V_T}}-1)$$

In [3]:

#Parameter value provided:
R_E = 2e3
T=27+273
C_L = 1e-6
V_A=150
beta=500
I_S=0.6e-12
k=1.38e-23
q=1.6e-19
VT=k*T/q
V_BE=0.6

#calculate IC, gm, etc
V_OUTDC=V1_DC-V_BE
I_C0=I_S*(np.exp(V_BE/VT)-1)

In [4]:
print('The DC current of Q1 is', f'{I_C0:0.5}', 'A')
print('The DC voltage at Vout is', f'{V_OUTDC:0.3}', 'V')

The DC current of Q1 is 0.0070591 A
The DC voltage at Vout is 3.06 V


Calculating DC operating point of Q1 by using iteration method:
$$V_{1,DC}-V_{BE}=I_{C0}R_{E}$$
$$I_{C0}=I_{S}(e^{\frac{V_{BE}}{V_T}}-1)$$
After solving the equation by iteration method, we found $V_{OUT,DC}=3.1V$, $I_{C0}=1.5mA$.

__b)__ Calculate the output resistance of the common-source stage (include $r_o$ of $M_1$), the input-resistance of the emitter-follower stage (*hint: this is not just $r_\pi$*!), and the output reistance of the emitter-follower.

*Solution:*

Output resistance of common-source stage: 
$$R_{OUT,CS}=r_{o,1}||R_D$$

Small signal modal of emitter follower:

<center><img src="img/HW3_b1.png" width=600></center>

Apply test voltage and current at emitter-follower input, and KCL at output (Ignore $r_o$):
$$\frac{v_{in}}{r_{\pi}}+g_{m}(v_{in}-v_{out})=\frac{v_{out}}{R_{E}}$$

$$i_{t}=\frac{v_{in}-v_{out}}{r_{\pi}}$$
Then input resistance of emitter-follower stage can be calculated:
$$R_{in2}=\frac{v_{in}}{i_t}=r_{\pi}+(1+\beta)R_{E}$$

Output resistance is related to output resistance of CS stage. Apply test voltage and current at emitter-follower output, and KCL at output (Ignore $r_o$):

<center><img src="img/HW3_b2.png" width=600></center>

$$i_{t}+g_{m}(v_{1}-v_{out})=\frac{v_{out}}{R_{E}}+\frac{v_{out}}{R_{\pi}+R_{OUT,CS}}$$
$$v_{1}=\frac{R_{OUT,CS}}{R_{OUT,CS}+r_{\pi}}$$

Then output resistance of emitter-follower stage can be calculated:
$$R_{out2}=\frac{v_{out}}{i_t}=\frac{R_{OUT,CS}}{\beta}+\frac{1}{g_{m}}$$

In [8]:
I_C0=1.5e-3 #Calculated by iteration
ro1=1/(lam*I_D0)
Routcs=R_D*ro1/(R_D+ro1)

gm2=I_C0/VT
rpi=beta/gm2
Rin2=rpi+(1+beta)*R_E
Rout2=1/gm2+Routcs/beta
print(gm2)
#print(I_C0)
#print(ro1)

0.05797101449275362


In [6]:
print('The output resistance of the common-source stage is', f'{Routcs:0.5}', 'Ohms')
print('The input resistance of the emitter-follower stage is', f'{Rin2:0.5}', 'Ohms')
print('The output resistance of the emitter follower stage is', f'{Rout2:0.5}', 'Ohms')

The output resistance of the common-source stage is 4933.7 Ohms
The input resistance of the emitter-follower stage is 1.0106e+06 Ohms
The output resistance of the emitter follower stage is 27.117 Ohms


__c)__ Calculate the small-signal voltage gain and 3dB bandwidth $\left(f_{3dB} = \dfrac{\omega_0}{2 \pi} \right)$ of the circuit assuming $C_L$ is the only capacitance.

*Solution:*

The voltage gain of the common-source stage: $A_1\approx-g_{m1}R_{OUT,CS}$

The voltage gain of the emitter follower stage: $A_{2}\approx 1$

The gain of two stages: $A\approx -g_{m1}R_{OUT,CS}$

Since $C_L$ is the only capacitance, this is the single pole system and the pole at the emitter-follower output is equal to the bandwidth.

$$f_{-3dB}=\frac{1}{2\pi R_{out2}C_{L}}$$

In [7]:
gm1=2*I_D0/(V_INDC-Vth)
Adc=-gm1*Routcs
f_3db=1/2/np.pi/Rout2/C_L
print(gm1)

0.002688000000000001


In [48]:
print('The small signal voltage gain is', f'{Adc:0.5}','V/V','which is',f'{20*np.log10(-Adc):0.4}', 'dB')
print('The 3dB bandwidth of the circuit is', f'{f_3db:0.3}', 'Hz')

The small signal voltage gain is -13.262 V/V which is 22.45 dB
The 3dB bandwidth of the circuit is 5.87e+03 Hz


<u>*Simulation*</u>

__d)__ Simulate the DC operating point (.op analysis) of the circuit in SPICE. Provide a screen capture of your circuit with all DC operating points labeled (right-click on the net and select *Place .op Data Label*).

Perform an AC simulation of the circuit and plot the frequency reponse. Indicate both the gain at $f = 10Hz$ and the 3dB bandwidth on the magnitude plot. 

Using either an AC analysis or DC Transfer analysis, simulate the DC input and output impedances of the circuit. Refer to the Zoom Cloud Recording following Lecture 3 if you are unsure of how to do this.

*Solution:*

Circuit schematic with all DC operaing points:

<center><img src="img/HW3_d1.png" width=1200></center>

Frequency response with gain and $3dB$ bandwidth:

<center><img src="img/HW3_d2.png" width=1500></center>

The simulated DC gain is $22.64dB$, 3dB bandwidth is $5.8kHz$. Both of them agree with the calculations.

Use AC analysis to simulate the input and output resistance at low frequency:

<center><img src="img/HW3_d3.png" width=1500></center>

<center><img src="img/HW3_d4.png" width=1500></center>

Input resistance ($R_{in}$) of the circuit is very large. Output resistance ($R_{out}$) is $27.9\Omega$, which agrees with the calculation. 

Use DC transfer to simulated the input and output resistance:

<center><img src="img/HW3_d5.png" width=800></center>

*A few words on hand calculations versus simulation results: It is expected that your simulation results will not agree perfectly with your calculations. This is actually the purpose of using SPICE for verification, in that more detailed and more accurate models are employed to provide a more accurate prediction of circuit performance than the simple models used for "back-of-the-envelope" analysis.That being said, our simple models should gives us enough guidance on making design decisions, particularly if simulation results are different from what we expect.*